In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!pip install tensorflow tensorflow-io matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 11.2 MB/s eta 0:00:00


In [21]:
! pip install -U scikit-learn

In [22]:
import os
import itertools
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

# Function to load and preprocess audio files
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

# Define paths to positive and negative data
POS = '/content/drive/MyDrive/aidataset/gunshot'
NEG = '/content/drive/MyDrive/aidataset/nongunshot'
pos = tf.data.Dataset.list_files(POS+'/*.wav')
neg = tf.data.Dataset.list_files(NEG+'/*.wav')

# Combine positive and negative datasets
positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
repeat_count = len(negatives) // len(positives)
remainder = len(negatives) % len(positives)
new_positives = positives.repeat(repeat_count)
new_positives = new_positives.concatenate(positives.take(remainder))
data = new_positives.concatenate(negatives)

# Preprocess the audio data
def preprocess(file_path, label):
    wav = load_wav_16k_mono(file_path)
    wav = wav[:40000]
    zero_padding = tf.zeros([40000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav], 0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)

    # Reshape the spectrogram to ensure it has the desired shape
    spectrogram = tf.reshape(spectrogram, (1241, 257, 1))

    # Resize spectrogram to match AlexNet input shape (227x227)
    spectrogram = tf.image.resize(spectrogram, (227, 227))

    # Repeat the single channel to create a three-channel image
    spectrogram_rgb = tf.repeat(spectrogram, repeats=3, axis=-1)

    return spectrogram_rgb, label

# Apply preprocessing to the dataset
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=5000)
data = data.batch(16)
data = data.prefetch(8)

train = data.take(130)
test = data.skip(130).take(30)
ttest = data.skip(160).take(20)

# Define AlexNet architecture
def AlexNet_Features(input_shape=(227, 227, 3)):
    model = Sequential([
        Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(256, (5, 5), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(384, (3, 3), activation='relu', padding='same'),
        Conv2D(384, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
    ])
    # Freeze the feature extraction layers (don't train them)
    for layer in model.layers:
        layer.trainable = False
    return model

# Create AlexNet feature extraction model
feature_extractor = AlexNet_Features()

# Extract features from the AlexNet model
features = []
labels = []
for spectrogram_rgb, label in data:
    extracted_features = feature_extractor.predict(spectrogram_rgb)
    features.append(extracted_features)
    labels.append(label)

features = np.concatenate(features, axis=0)
labels = np.concatenate(labels, axis=0)

test_features = []
test_labels = []
for spectrogram_rgb, label in test:
    extracted_features = feature_extractor.predict(spectrogram_rgb)
    test_features.append(extracted_features)
    test_labels.append(label)

test_features = np.concatenate(test_features, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Function to calculate information gain for each feature
def information_gain(X, y, feature_index):
    total_entropy = entropy(y)
    unique_values, counts = np.unique(X[:, feature_index], return_counts=True)
    weighted_entropy = 0
    for value, count in zip(unique_values, counts):
        subset_y = y[X[:, feature_index] == value]
        weighted_entropy += (count / len(y)) * entropy(subset_y)
    return total_entropy - weighted_entropy

# Function to calculate entropy
def entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    return -np.sum(probabilities * np.log2(probabilities))

# Function to find the feature with the highest information gain
def find_root_node(features, labels):
    num_features = features.shape[1]
    best_information_gain = -1
    best_feature_index = -1
    for i in range(num_features):
        ig = information_gain(features, labels, i)
        if ig > best_information_gain:
            best_information_gain = ig
            best_feature_index = i
    return best_feature_index

# Identify the root node feature
root_node_index = find_root_node(features, labels)
print("Root node feature index:", root_node_index)

# Train Decision Tree classifier using the identified root node feature
clf_decision_tree = DecisionTreeClassifier(criterion="entropy", random_state=100)
clf_decision_tree.fit(features[:, root_node_index].reshape(-1, 1), labels)

# Evaluate the model
test_predictions = clf_decision_tree.predict(test_features[:, root_node_index].reshape(-1, 1))
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy of Decision Tree classifier:", accuracy)




1/1 [==============================] - 1s 693ms/step
Root node feature index: 35
Accuracy of Decision Tree classifier: 1.0


In [25]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

precision = precision_score(test_labels, test_predictions)
print("P of Decision Tree classifier:", precision)
f1 = f1_score(test_labels, test_predictions)
print("f of Decision Tree classifier:", f1)
recall = recall_score(test_labels, test_predictions)
print("r of Decision Tree classifier:", recall)

P of Decision Tree classifier: 1.0
f of Decision Tree classifier: 1.0
r of Decision Tree classifier: 1.0


In [27]:
num_features = features.shape[1]
print("Number of features:", num_features)

Number of features: 4096
